In [14]:
import pandas as pd
import dask.dataframe as dd
from dask.dataframe.utils import make_meta
from dask.multiprocessing import get

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
mpl.rcParams.update({
    'font.size'           : 16.0,
    'axes.titlesize'      : 'large',
    'axes.labelsize'      : 'medium',
    'xtick.labelsize'     : 'small',
    'ytick.labelsize'     : 'small',
    'legend.fontsize'     : 'small',})
get_ipython().magic('matplotlib inline')

In [ ]:
# df = pd.read_pickle('df_active.pkl')
# print(df.shape)
# df.tail()

In [ ]:
# ddf = dd.from_pandas(df, chunksize=5000)
# col_df = pd.DataFrame(pd.Series(ddf.columns).isin(cols))
# col_df[col_df[0]==True].index

In [3]:
cols = [  0,   1,  10,  23,  39,  40,  41,  47,  50,  51,  54,  55,  58,
             59,  62,  63,  66,  67,  70,  71,  74,  75,  78,  79,  82,  83,
             86,  87,  90,  91,  94,  95,  98,  99, 102, 103, 106, 307, 308]

In [4]:
raw = dd.read_csv("s3://gschoolcapstone/npidata_20050523-20170813.csv", usecols=cols, dtype='object')

In [5]:
raw.head()

,NPI,Entity Type Code,Provider Credential Text,Provider Business Mailing Address State Name,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,...,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Is Sole Proprietor,Is Organization Subpart
0,1679576722,1,M.D.,NE,NaN,NaN,M,207X00000X,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
1,1588667638,1,MD,FL,NaN,NaN,M,207RC0000X,Y,207RC0000X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,1497758544,2,NaN,NC,NaN,NaN,NaN,251G00000X,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,1306849450,1,M.D.,TX,NaN,NaN,M,2085R0202X,Y,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,1215930367,1,M.D.,TX,NaN,NaN,M,174400000X,N,207RH0003X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [15]:
active = raw['Entity Type Code'].notnull()
df = raw[active].reset_index()

In [16]:
code_df = pd.read_csv('s3n://gschoolcapstone/nucc_taxonomy_171.csv')
code_cols = [col for col in df.columns if col.startswith('Healthcare Provider Taxonomy Code_')]

In [20]:
def get_prim_col_name(row):
    b = (row == 'Y')
    n = b.argmax()[44:]
    col = "Healthcare Provider Taxonomy Code_"+n
    return df[col][row.name]

In [ ]:
df['Primary Tax Code'] = df[[col for col in df.columns \
                             if col.startswith('Healthcare Provider Primary ')]]\
                             .apply(get_prim_col_name, axis=1)

# df['Primary Tax Code Cat'] = df['Primary Tax Code'].astype('category', categories=code_df.Code, ordered=False)
# df['Primary Tax Code Cat'] = df['Primary Tax Code Cat'].cat.codes

In [ ]:
HPPcols = [col for col in df.columns if col.startswith('Healthcare Provider Primary ')]
df.map_partitions(get_prim_col_name, columns=HPPcols).compute(get=get)